In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import time

# Paths and stuff
import os
import sys

sys.path.append('/home/shreyas/pySICOPOLIS/src')
from pySICOPOLIS import *

In [2]:
ds_grl40_bm5_paleo17a_CT4_BH0_m21ka_pkp_init = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka0005.nc')
ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka0006.nc')

# Manually verify gradients in sicopolis_validation_genctrl_nolog match sicopolis_validation. A quick check of SUM(varb) for each var should suffice.
# Once you are confident in sicopolis_validation_genctrl_nolog, use it to verify sicopolis_validation_genctrl_prof_real*.
# The ad_input*.nc files for sicopolis_validation_genctrl_prof_real* are generated in their respective mini directories.
sicopolis_validation_genctrl_nolog_dir = '/home/shreyas/update_to_develop_sicopolis/sicopolis_validation_genctrl_nolog'
sicopolis_validation_genctrl_prof_realSVC_dir = '/home/shreyas/update_to_develop_sicopolis/sicopolis_validation_genctrl_prof_realSVC'
sicopolis_validation_genctrl_prof_realACBM5ZLC_dir = '/home/shreyas/update_to_develop_sicopolis/sicopolis_validation_genctrl_prof_realACBM5ZLC'

H_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/bm5_data_40kms.nc")
age_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/age_data_40kms.nc")

H_data = H_data_40["H"].data
H_uncert_data = H_data_40["H_uncert"].data
zl_uncert_data = H_data_40["zl_uncert"].data

age_c_data = age_data_40["age_c"].data
age_c_uncert_data = age_data_40["age_c_uncert"].data

mask_age_c = np.zeros(age_c_data.shape)
for kc in range(age_c_data.shape[0]):
    for j in range(age_c_data.shape[1]):
        for i in range(age_c_data.shape[2]):
            if age_c_uncert_data[kc, j, i] > 0 and age_c_data[kc, j, i] >= 0 and age_c_data[kc, j, i] <= 134000 and H_data[j, i] >= 2000.0:
                mask_age_c[kc, j, i] = 1.0

In [3]:
# Compile all executables before this block, so that some of them can run for this block.
# Since all we need is fake sampling to get all the netcdf files including prior files, no need to change simulation name I think. 
# The generated input files should work for both cases as long as the same variables are activated in the same order.
# NTDAMAX will have to be set to 5 to make it work though and for direct comparison with validation_genctrl_prof_real* setups.
simulation = 'grl40_bm5_paleo17a_CT4_BH0_AC_BM5_ZLC_m11ka_pkp'
# simulation = 'grl40_bm5_paleo17a_nudged_CT4_BH0_m21ka_pkp'

dict_sico_out_folder_prefixes = {"nodiff": "N",
                                 "tlm": "FORWARD",
                                 "adj": "ADJOINT",
                                 "tlm_action": "FORWARDHESSACTION",
                                 "adj_action": "ADJHESSACTION"}
dict_ad_exec_cmds_suffixes = {"nodiff": "nodiff",
                              "tlm": "forward",
                              "adj": "adjoint",
                              "tlm_action": "forwardhessaction",
                              "adj_action": "adjointhessaction"}
dict_ad_log_file_suffixes = {"nodiff": "nodiff",
                             "tlm": "tlm",
                             "adj": "adj",
                             "tlm_action": "tlm_hessaction",
                             "adj_action": "adj_hessaction"}
dict_ad_nc_suffixes = {"nodiff": "nodiff",
                       "tlm": "tlm",
                       "adj": "adj",
                       "tlm_action": "tlm_hessaction",
                       "adj_action": "adj_hessaction"}

KCMAX = 80
exp_sigma_level = dataCleaner.exp_sigma_level(zeta = np.arange(0,1+1./KCMAX,1./KCMAX),
                                              exponent = 2.0)
KRMAX = 40
zeta_r = np.arange(0.,1. + 1.0/KRMAX, 1.0/KRMAX)
xModel40       = np.arange(-72.,97.,4.0)*10
yModel40       = np.arange(-345.,-56.,4.0)*10
# Set NTDAMAX to 5 to match mini setups from which the validation_genctrl_prof_real* setups derive their ad_input_*.nc files.
time_ad = np.arange(6, dtype=float)
IMAX = xModel40.shape[0]-1
JMAX = yModel40.shape[0]-1
NTDAMAX = time_ad.shape[0]-1

dict_og_params_fields_vals_nolog = {"xx_c_slide_init": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_delta_tda_const": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_c_dis_da": 0.0,
                                    "xx_q_geo": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_gamma_s": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_s_stat": 0.0,
                                    "xx_beta1": 0.0,
                                    "xx_beta2": 0.0,
                                    "xx_Pmax": 0.0,
                                    "xx_mu": 0.0,
                                    "xx_RHO_A": 0.0,
                                    "xx_time_lag_asth": 0.0,
                                    "xx_flex_rig_lith": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_p_weert": 0.0,
                                    "xx_q_weert": 0.0,
                                    "xx_enh_fact_da_dummy2d_scalar": 0.0,
                                    "xx_enh_intg_da_dummy2d_scalar": 0.0,
                                    "xx_n_glen_da_dummy2d_scalar": 0.0,
                                    "xx_zs": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_zl": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_zl0": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_zb": np.zeros((JMAX+1, IMAX+1), dtype=float),
                                    "xx_temp_c": np.zeros((KCMAX+1, JMAX+1, IMAX+1), dtype=float),
                                    "xx_omega_c": np.zeros((KCMAX+1, JMAX+1, IMAX+1), dtype=float),
                                    "xx_age_c": np.zeros((KCMAX+1, JMAX+1, IMAX+1), dtype=float),
                                    "xx_temp_r": np.zeros((KRMAX+1, JMAX+1, IMAX+1), dtype=float),
                                    "xx_delta_tda": np.zeros((NTDAMAX+1, JMAX+1, IMAX+1), dtype=float)}
dict_prior_params_fields_vals_nolog = dict_og_params_fields_vals_nolog.copy()

dict_params_fields_num_dims = {"xx_c_slide_init": "2D",
                               "xx_delta_tda_const": "2D",
                               "xx_c_dis_da": "2D",
                               "xx_q_geo": "2D",
                               "xx_gamma_s": "2D",
                               "xx_s_stat": "2D",
                               "xx_beta1": "2D",
                               "xx_beta2": "2D",
                               "xx_Pmax": "2D",
                               "xx_mu": "2D",
                               "xx_RHO_A": "2D",
                               "xx_time_lag_asth": "2D",
                               "xx_flex_rig_lith": "2D",
                               "xx_p_weert": "2D",
                               "xx_q_weert": "2D",
                               "xx_enh_fact_da_dummy2d_scalar": "2D",
                               "xx_enh_intg_da_dummy2d_scalar": "2D",
                               "xx_n_glen_da_dummy2d_scalar": "2D",
                               "xx_zs": "2D",
                               "xx_zl": "2D",
                               "xx_zl0": "2D",
                               "xx_zb": "2D",
                               "xx_temp_c": "3D",
                               "xx_omega_c": "3D",
                               "xx_age_c": "3D",
                               "xx_temp_r": "3DR",
                               "xx_delta_tda": "2DT"}

dict_params_coords = {"time_ad": time_ad,
                      "zeta_c": exp_sigma_level,
                      "zeta_r": zeta_r,
                      "y": yModel40,
                      "x": xModel40}

dict_params_attrs_type = {"xx_c_slide_init": "nodiff",
                          "xx_delta_tda_const": "nodiff",
                          "xx_c_dis_da": "nodiff",
                          "xx_q_geo": "nodiff",
                          "xx_gamma_s": "nodiff",
                          "xx_s_stat": "nodiff",
                          "xx_beta1": "nodiff",
                          "xx_beta2": "nodiff",
                          "xx_Pmax": "nodiff",
                          "xx_mu": "nodiff",
                          "xx_RHO_A": "nodiff",
                          "xx_time_lag_asth": "nodiff",
                          "xx_flex_rig_lith": "nodiff",
                          "xx_p_weert": "nodiff",
                          "xx_q_weert": "nodiff",
                          "xx_enh_fact_da_dummy2d_scalar": "nodiff",
                          "xx_enh_intg_da_dummy2d_scalar": "nodiff",
                          "xx_n_glen_da_dummy2d_scalar": "nodiff",
                          "xx_zs": "nodiff",
                          "xx_zl": "nodiff",
                          "xx_zl0": "nodiff",
                          "xx_zb": "nodiff",
                          "xx_temp_c": "nodiff",
                          "xx_omega_c": "nodiff",
                          "xx_age_c": "nodiff",
                          "xx_temp_r": "nodiff",
                          "xx_delta_tda": "nodiff"}

dict_params_fields_or_scalars = {"xx_c_slide_init": "field",
                                 "xx_delta_tda_const": "field",
                                 "xx_c_dis_da": "scalar",
                                 "xx_q_geo": "field",
                                 "xx_gamma_s": "field",
                                 "xx_s_stat": "scalar",
                                 "xx_beta1": "scalar",
                                 "xx_beta2": "scalar",
                                 "xx_Pmax": "scalar",
                                 "xx_mu": "scalar",
                                 "xx_RHO_A": "scalar",
                                 "xx_time_lag_asth": "scalar",
                                 "xx_flex_rig_lith": "field",
                                 "xx_p_weert": "scalar",
                                 "xx_q_weert": "scalar",
                                 "xx_enh_fact_da_dummy2d_scalar": "scalar",
                                 "xx_enh_intg_da_dummy2d_scalar": "scalar",
                                 "xx_n_glen_da_dummy2d_scalar": "scalar",
                                 "xx_zs": "field",
                                 "xx_zl": "field",
                                 "xx_zl0": "field",
                                 "xx_zb": "field",
                                 "xx_temp_c": "field",
                                 "xx_omega_c": "field",
                                 "xx_age_c": "field",
                                 "xx_temp_r": "field",
                                 "xx_delta_tda": "field"}

year2sec = 3.1556925445e+07
dict_masks_observables = {"H": H_uncert_data**(-2),
                          "zl": zl_uncert_data**(-2),
                          "age_c": mask_age_c*(age_c_uncert_data*year2sec)**(-2)}

dict_prior_sigmas_nolog = {"xx_c_slide_init": 1.5,
                     "xx_delta_tda_const": 0.1,
                     "xx_c_dis_da": 1000.0,
                     "xx_q_geo": 100.0,
                     "xx_gamma_s": 0.01,
                     "xx_s_stat": 1.0,
                     "xx_beta1": 1.0,
                     "xx_beta2": 1.0,
                     "xx_Pmax": 1.0,
                     "xx_mu": 1.0,
                     "xx_RHO_A": 100.0,
                     "xx_time_lag_asth": 100.0,
                     "xx_flex_rig_lith": 1.e+24,
                     "xx_p_weert": 1.0,
                     "xx_q_weert": 1.0,
                     "xx_enh_fact_da_dummy2d_scalar": 0.3,
                     "xx_enh_intg_da_dummy2d_scalar": 0.3,
                     "xx_n_glen_da_dummy2d_scalar": 0.2,
                     "xx_zs": 300.0,
                     "xx_zl": 300.0,
                     "xx_zl0": 300.0,
                     "xx_zb": 300.0,
                     "xx_temp_c": 0.5,
                     "xx_omega_c": 0.01,
                     "xx_age_c": 100.0,
                     "xx_temp_r": 0.5,
                     "xx_delta_tda": 0.1}

dict_prior_gammas = {"xx_c_slide_init": 1.0,
                     "xx_delta_tda_const": 1.0,
                     "xx_c_dis_da": 0.0,
                     "xx_q_geo": 1.0,
                     "xx_gamma_s": 1.0,
                     "xx_s_stat": 0.0,
                     "xx_beta1": 0.0,
                     "xx_beta2": 0.0,
                     "xx_Pmax": 0.0,
                     "xx_mu": 0.0,
                     "xx_RHO_A": 0.0,
                     "xx_time_lag_asth": 0.0,
                     "xx_flex_rig_lith": 1.0,
                     "xx_p_weert": 0.0,
                     "xx_q_weert": 0.0,
                     "xx_enh_fact_da_dummy2d_scalar": 0.0,
                     "xx_enh_intg_da_dummy2d_scalar": 0.0,
                     "xx_n_glen_da_dummy2d_scalar": 0.0,
                     "xx_zs": 1.0,
                     "xx_zl": 1.0,
                     "xx_zl0": 1.0,
                     "xx_zb": 1.0,
                     "xx_temp_c": 0.0,
                     "xx_omega_c": 0.0,
                     "xx_age_c": 0.0,
                     "xx_temp_r": 0.0,
                     "xx_delta_tda": 1.0}

dict_prior_deltas = {"xx_c_slide_init": 0.001,
                     "xx_delta_tda_const": 0.001,
                     "xx_c_dis_da": 0.001,
                     "xx_q_geo": 0.001,
                     "xx_gamma_s": 0.001,
                     "xx_s_stat": 0.001,
                     "xx_beta1": 0.001,
                     "xx_beta2": 0.001,
                     "xx_Pmax": 0.001,
                     "xx_mu": 0.001,
                     "xx_RHO_A": 0.001,
                     "xx_time_lag_asth": 0.001,
                     "xx_flex_rig_lith": 0.001,
                     "xx_p_weert": 0.001,
                     "xx_q_weert": 0.001,
                     "xx_enh_fact_da_dummy2d_scalar": 0.001,
                     "xx_enh_intg_da_dummy2d_scalar": 0.001,
                     "xx_n_glen_da_dummy2d_scalar": 0.001,
                     "xx_zs": 0.001,
                     "xx_zl": 0.001,
                     "xx_zl0": 0.001,
                     "xx_zb": 0.001,
                     "xx_temp_c": 0.001,
                     "xx_omega_c": 0.001,
                     "xx_age_c": 0.001,
                     "xx_temp_r": 0.001,
                     "xx_delta_tda": 0.001}

list_fields_to_ignore = None
MAX_ITERS_SOR = 50
OMEGA_SOR = 1.5

## Use only when you want to generate the ad_input files for validation_genctrl_nolog.
# _ = optim.DataAssimilation(sicopolis_validation_genctrl_nolog_dir, simulation,
#                             dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
#                             dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
#                             dict_og_params_fields_vals_nolog, dict_prior_params_fields_vals_nolog, dict_params_fields_num_dims, 
#                             dict_params_coords, dict_params_attrs_type, dict_params_fields_or_scalars, dict_masks_observables,
#                             dict_prior_sigmas_nolog, dict_prior_gammas, dict_prior_deltas,
#                             MAX_ITERS_SOR, OMEGA_SOR, list_fields_to_ignore, False, None, "/scratch2/shreyas/ad_validation_genctrl_nolog", 10, None)

In [4]:
### GO TO THE TERMINAL AND RUN THE ADJOINTS IN ALL DIRS NOW BEFORE RUNNING THE REST OF THIS NOTEBOOK.

In [5]:
ds_validation_genctrl_nolog_SVC = xr.open_dataset(sicopolis_validation_genctrl_nolog_dir + "/src/subroutines/tapenade/ad_io/ad_output_SVC_nudged_m21ka_pkp_adj_genctrl_nooptim_noprof.nc")
ds_validation_genctrl_nolog_ACBM5ZLC = xr.open_dataset(sicopolis_validation_genctrl_nolog_dir + "/src/subroutines/tapenade/ad_io/ad_output_AC_BM5_ZLC_m11ka_pkp_adj_genctrl_nooptim_noprof.nc")
ds_validation_genctrl_prof_realSVC = xr.open_dataset(sicopolis_validation_genctrl_prof_realSVC_dir + "/src/subroutines/tapenade/ad_io/ad_output_SVC_nudged_m21ka_pkp_adj_genctrl_nooptim_noprof_realSVC.nc")
ds_validation_genctrl_prof_realACBM5ZLC = xr.open_dataset(sicopolis_validation_genctrl_prof_realACBM5ZLC_dir + "/src/subroutines/tapenade/ad_io/ad_output_AC_BM5_ZLC_m11ka_pkp_adj_genctrl_optim_noprof_realACBM5ZLC.nc")

In [6]:
def compare(ds_nolog, ds_genctrl_prof, var_name, var_log10_value = None, bool_log = False):
    if bool_log and var_log10_value is None:
        return ValueError("If bool_log is True, var_log10_value must be specified.")

    varb_validation_genctrl_nolog = ds_nolog["xx_" + var_name + "b"].data
    if not bool_log:
        varb_validation_genctrl_prof = ds_genctrl_prof["xx_" + var_name + "b"].data
    else:
        varb_validation_genctrl_prof = 10**(-var_log10_value)*ds_genctrl_prof["xx_" + var_name + "b"].data/np.log(10)
        
    print(np.sum(varb_validation_genctrl_nolog), 
          np.sum(varb_validation_genctrl_prof),
          np.sum(np.abs(varb_validation_genctrl_nolog)), 
          np.sum(np.abs(varb_validation_genctrl_prof)),
          np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),
          np.sum((varb_validation_genctrl_nolog == 0.0) & (varb_validation_genctrl_prof != 0.0)),
          np.sum((varb_validation_genctrl_nolog != 0.0) & (varb_validation_genctrl_prof == 0.0)))

In [7]:
log_c_slide_init = np.log10(0.85)*np.ones((JMAX+1, IMAX+1))
compare(ds_validation_genctrl_nolog_SVC, ds_validation_genctrl_prof_realSVC, "c_slide_init", log_c_slide_init, True)

14550.730486413948 14545.566345827472 25010.513737851357 25014.355478553814 6.772278696676182e-05 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [8]:
data = ds_grl40_bm5_paleo17a_CT4_BH0_m21ka_pkp_init['q_geo'].data
log_q_geo = np.where(data > 0, np.log10(data), -10)
compare(ds_validation_genctrl_nolog_SVC, ds_validation_genctrl_prof_realSVC, "q_geo", log_q_geo, True)

57363.03989926748 57372.263903937404 88461.07139749473 88469.25830458342 -0.0 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [9]:
log_mu = np.log10(9.7155)
compare(ds_validation_genctrl_nolog_SVC, ds_validation_genctrl_prof_realSVC, "mu", log_mu, True)

64.24935198699222 64.25655351568915 93.75862189086232 93.76623670643959 3.573796260614159e-05 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [10]:
log_p_weert = np.log10(3.0)
compare(ds_validation_genctrl_nolog_SVC, ds_validation_genctrl_prof_realSVC, "p_weert", log_p_weert, True)

-4205.284563676415 -4205.460213723953 5962.643939407653 5962.831236077991 -0.0 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [11]:
log_q_weert = np.log10(2.0)
compare(ds_validation_genctrl_nolog_SVC, ds_validation_genctrl_prof_realSVC, "q_weert", log_q_weert, True)

6085.892590423664 6087.696150098912 12038.106417314557 12039.722742220332 1.423586519266884e-05 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [12]:
log_enh_fact_da_dummy2d_scalar = np.log10(3.0)
compare(ds_validation_genctrl_nolog_SVC, ds_validation_genctrl_prof_realSVC, "enh_fact_da_dummy2d_scalar", log_enh_fact_da_dummy2d_scalar, True)

2825.030836365716 2827.7099991886203 2825.030836365716 2827.7099991886203 0.0009483658685827123 0 0


In [13]:
log_n_glen_da_dummy2d_scalar = np.log10(3.0)
compare(ds_validation_genctrl_nolog_SVC, ds_validation_genctrl_prof_realSVC, "n_glen_da_dummy2d_scalar", log_n_glen_da_dummy2d_scalar, True)

-4800.696575587761 -4801.964385641366 4800.696575587761 4801.964385641366 0.00026408876996164565 0 0


In [14]:
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "delta_tda_const", None, False)

-34770.60953800539 -34770.607197638004 59127.82599048865 59122.949701021585 0.0009414976823600707 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [15]:
log_c_dis_da = np.log10(15659.0)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "c_dis_da", log_c_dis_da, True)

-3.103756431276544 -3.1036194695836077 3.6688347511782493 3.668442472862811 0.0004737800126539682 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [16]:
log_gamma_s = np.log10(0.070458)*np.ones((JMAX+1, IMAX+1))
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "gamma_s", log_gamma_s, True)

11863.20222088675 11838.165336388949 253052.45877444284 253046.14108467917 0.0011612953429731565 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [17]:
log_s_stat = np.log10(5.0)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "s_stat", log_s_stat, True)

-24849.29308419575 -24848.033902601026 33997.92789311879 33992.851311550236 0.0009665837500654195 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [18]:
log_beta1 = np.log10(2.73)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "beta1", log_beta1, True)

-7875.697814615456 -7875.073026085394 11165.869403320845 11164.31304531297 0.0005683923605215302 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [19]:
log_beta2 = np.log10(7.28)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "beta2", log_beta2, True)

-12394.28640129987 -12393.62639654693 15622.346857000844 15619.987578443737 0.0009739253932194991 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [20]:
log_Pmax = np.log10(0.6)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "Pmax", log_Pmax, True)

65288.01044980501 65284.08950891373 86137.64258829784 86130.09729830586 0.0010567918895105777 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [21]:
log_RHO_A = np.log10(3300.0)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "RHO_A", log_RHO_A, True)

8.209646012717723 8.20940993031591 27.60136248736306 27.60113334293088 0.0077115674633290165 0 0


In [22]:
log_time_lag_asth = np.log10(3000.0)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "time_lag_asth", log_time_lag_asth, True)

-1.6943351617325533 -1.6942474977722117 3.2273019234129774 3.2271405595406804 0.006385258114273817 0 0


In [23]:
log_flex_rig_lith = np.log10(1.e25)*np.ones((JMAX+1, IMAX+1))
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "flex_rig_lith", log_flex_rig_lith, True)

2.70918318419685e-21 2.709105277004251e-21 9.10844962082981e-21 9.108374003167194e-21 0.0077115674633287945 0 0


In [24]:
log_enh_intg_da_dummy2d_scalar = np.log10(1.0)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "enh_intg_da_dummy2d_scalar", log_enh_intg_da_dummy2d_scalar, True)

-17405.88951467569 -17406.093391302034 17405.88951467569 17406.093391302034 1.1713082872066849e-05 0 0


In [25]:
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "zs", None, False)

32.75011848865601 32.74540092591592 54.986191748752546 54.98729727115624 1.910190496334785e-05 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [26]:
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "zl", None, False)

5.766042293577237 5.785929913692861 185.08687345940015 185.0949587964675 0.019287296580427826 0 0


In [27]:
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "temp_c", None, False)

-3183.1670501515937 -3184.2337516752914 5856.132690747058 5857.82832269005 0.006518351667576416 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [28]:
data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['omega_c'].data
log_omega_c = np.where(data > 0, np.log10(data), -10)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "omega_c", log_omega_c, True)

/tmp/ipykernel_2145777/144168397.py:2: RuntimeWarning: divide by zero encountered in log10
  log_omega_c = np.where(data > 0, np.log10(data), -10)


-538836.4406157025 -539018.9949423469 993110.2923554329 993401.8478193342 0.006658150156317655 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [29]:
data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['age_c'].data
log_age_c = np.where(data > 0, np.log10(data), -10)
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "age_c", log_age_c, True)

6.656530397531861 6.656510538334067 9.557680410937694 9.557659582997825 1.0565239023874184e-05 0 0


/tmp/ipykernel_2145777/3401026808.py:2: RuntimeWarning: divide by zero encountered in log10
  log_age_c = np.where(data > 0, np.log10(data), -10)
/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [30]:
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "temp_r", None, False)

-3496.3633617257055 -3496.5560682195855 7053.832445935527 7053.746291077227 0.0023133975856513 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),


In [31]:
compare(ds_validation_genctrl_nolog_ACBM5ZLC, ds_validation_genctrl_prof_realACBM5ZLC, "delta_tda", None, False)

9.094947017729282e-13 1.8189894035458565e-12 54810.88181347219 54808.2608272904 0.0006581971303309953 0 0


/tmp/ipykernel_2145777/308923850.py:15: RuntimeWarning: invalid value encountered in divide
  np.nanmax((np.abs(varb_validation_genctrl_prof/varb_validation_genctrl_nolog) - 1.0) * (np.abs(varb_validation_genctrl_nolog) > 0.1*np.max(np.abs(varb_validation_genctrl_nolog)))),
